In [1]:
import numpy as np
import pandas as pd
#from tqdm import tqdm

from numpy.linalg import norm
import json


In [2]:
with open('/Users/s0c02nj/Documents/covid-19/notebooks/output_fra.jsonl') as fin:
    output_jsonl = fin.read()

In [9]:
def get_sent_embed(output_jsonl) :
    
    #We will run the model and get the outputs
    json_lines = output_jsonl.split('\n')
    
    #Removing the blank strings
    json_lines =  list(filter(None,json_lines))
    
    #getting the dimensions & getting the output of the query
    line_q = json.loads(json_lines[0])
    embed_size = len(line_q['features'][0]['layers'][0]['values'])
    
    #Temp list for saving the tokens
    token_temp_q = []
    
    #array for saving the embeddings
    feat_embed_q =  np.array(line_q['features'][0]['layers'][0]['values'])
    
    #Getting the final df
    df_query = pd.DataFrame()
    
    for j,feature in enumerate(line_q['features']):
        token_temp_q.append(feature['token'])


    #final_output_embeddings
    tokens_query = ' '.join(token_temp_q[1:len(token_temp_q)-1])

    #final query dataframe
    df_query['documents'] = [tokens_query]
    df_query['embedding'] = [feat_embed_q]
    
    
    #--------------------------------------- answers ----------------------------------------------#
    
    
    #Defining the lists
    sent_embed = []
    tokens = []
    
    #Getting the final df
    df_ans = pd.DataFrame()
    
    #Running for the sentence
    for i in range(1,len(json_lines)):
        line = json.loads(json_lines[i])        
    
        feat_embed = np.array(line['features'][0]['layers'][0]['values'])
        
        #Temp list for saving the tokens
        token_temp = []
        
        for j,feature in enumerate(line['features']):
            token_temp.append(feature['token'])
            
        
        #sanity checks
        if feat_embed.sum() == 0 :
            print ('Check_model')
        
        #final_output_embeddings
        sent_embed.append(feat_embed)
        tokens.append(' '.join(token_temp[1:len(token_temp)-1]))
        
         
        
    df_ans['documents'] = tokens
    df_ans['embedding'] = sent_embed
    
    return df_query, df_ans
        

In [10]:
df_query, df_ans = get_sent_embed(output_jsonl)

In [14]:
def get_cosine_sim(output_jsonl) :
    
    #We will run the model and get the outputs
    
    #Getting the dataframes
    df_query ,df_ans = get_sent_embed(output_jsonl)
    
    #Query embedding
    query_embed = df_query['embedding'].values[0]
    query_embed_norm = query_embed/norm(query_embed)
    
    #Answers embedding
    list_embed = df_ans['embedding'].tolist()
    
    #getting the answer embedding
    ans_embed =  np.stack(list_embed, axis=0)
    ans_embed_norm = ans_embed/norm(ans_embed,axis=1,keepdims= True)
    
    #find the cosine similarity
    cos_sim = np.dot(ans_embed_norm,query_embed_norm)
    
    return cos_sim.argsort()[::-1],np.sort(cos_sim)[::-1]

In [15]:
ind,score = get_cosine_sim(output_jsonl)

In [16]:
ind

array([0, 1, 2, 3])

In [17]:
score

array([0.97327895, 0.96882837, 0.91632218, 0.44585593])